In [13]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"]=userdata.get('OpenAI-Key')

In [8]:
!pip install openai
!openai migrate

Retrieving Grit CLI metadata from https://api.keygen.sh/v1/accounts/custodian-dev/artifacts/marzano-linux-x64



Analyzing Finding files                                                         
Processed 0 files and found 0 matches


In [3]:
# Importing the necessary Python libraries
import os
import json
import openai

In [14]:
from openai import OpenAI

client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)

In [4]:
about_me = 'Hello! My name is David Hundley. I am a principal machine learning engineer at State Farm. I enjoy learning about AI and teaching what I learn back to others. I have two daughters. I drive a Tesla Model 3, and my favorite video game series is The Legend of Zelda.'

In [5]:
# Engineering a prompt to extract as much information from "About Me" as a JSON object
about_me_prompt = f'''
Please extract information as a JSON object. Please look for the following pieces of information.
Name
Job title
Company
Number of children as a single number
Car make
Car model
Favorite video game series

This is the body of text to extract the information from:
{about_me}
'''

In [21]:
# Getting the response back from ChatGPT (gpt-3.5-turbo)
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': about_me_prompt}]
)

In [29]:
print(completion.model_dump_json(indent=2))

{
  "id": "chatcmpl-8nj3jt0WWpzc21IAaRw4CL1WWTC0S",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "{\n  \"Name\": \"David Hundley\",\n  \"Job title\": \"Principal machine learning engineer\",\n  \"Company\": \"State Farm\",\n  \"Number of children\": 2,\n  \"Car make\": \"Tesla\",\n  \"Car model\": \"Model 3\",\n  \"Favorite video game series\": \"The Legend of Zelda\"\n}",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1706861971,
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 70,
    "prompt_tokens": 122,
    "total_tokens": 192
  }
}


In [27]:
tfed_response = completion.model_dump_json(indent=2)

In [45]:
print(json.loads(tfed_response)['choices'][0]['message']['content'])

{
  "Name": "David Hundley",
  "Job title": "Principal machine learning engineer",
  "Company": "State Farm",
  "Number of children": 2,
  "Car make": "Tesla",
  "Car model": "Model 3",
  "Favorite video game series": "The Legend of Zelda"
}


In [48]:
# Loading the response as a JSON object
json_response = json.loads(tfed_response)['choices'][0]['message']['content']
# print(type(json_response))
json.loads(json_response)

<class 'str'>


{'Name': 'David Hundley',
 'Job title': 'Principal machine learning engineer',
 'Company': 'State Farm',
 'Number of children': 2,
 'Car make': 'Tesla',
 'Car model': 'Model 3',
 'Favorite video game series': 'The Legend of Zelda'}

In [49]:
# Defining our initial extract_person_info function
def extract_person_info(name, job_title, num_children):
    '''
    Prints basic "About Me" information

    Inputs:
        name (str): Name of the person
        job_title (str): Job title of the person
        num_chilren (int): The number of children the parent has.
    '''

    print(f'This person\'s name is {name}. Their job title is {job_title}, and they have {num_children} children.')


In [51]:
# Defining how we want ChatGPT to call our custom functions
my_custom_functions = [
    {
        'name': 'extract_person_info',
        'description': 'Get "About Me" information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'job_title': {
                    'type': 'string',
                    'description': 'Job title of the person'
                },
                'num_children': {
                    'type': 'integer',
                    'description': 'Number of children the person is a parent to'
                }
            }
        }
    }
]

In [55]:
# Getting the response back from ChatGPT (gpt-3.5-turbo)
openai_response = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': about_me}],
    functions = my_custom_functions,
    function_call = 'auto'
)

response = openai_response.model_dump_json(indent=2)
print(response)

{
  "id": "chatcmpl-8njXvENfoMcTXwgtUixosRFZTWBm3",
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "role": "assistant",
        "function_call": {
          "arguments": "{\n  \"name\": \"David Hundley\",\n  \"job_title\": \"Principal Machine Learning Engineer\",\n  \"num_children\": 2\n}",
          "name": "extract_person_info"
        },
        "tool_calls": null
      }
    }
  ],
  "created": 1706863843,
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 37,
    "prompt_tokens": 147,
    "total_tokens": 184
  }
}


In [56]:
# Getting the response back from ChatGPT (gpt-3.5-turbo)
openai_response = client.chat.completions.create(
    model = 'gpt-3.5-turbo',
    messages = [{'role': 'user', 'content': 'How tall is the Eiffel Tower?'}],
    functions = my_custom_functions,
    function_call = 'auto'
)

response = openai_response.model_dump_json(indent=2)
print(response)

{
  "id": "chatcmpl-8njYgrkmOoO6pxPKPBiCrxMmtMpEp",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "The Eiffel Tower is approximately 330 meters (1,083 feet) tall.",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1706863890,
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 19,
    "prompt_tokens": 97,
    "total_tokens": 116
  }
}


In [57]:
# Defining a function to extract only vehicle information
def extract_vehicle_info(vehicle_make, vehicle_model):
    '''
    Prints basic vehicle information

    Inputs:
        - vehicle_make (str): Make of the vehicle
        - vehicle_model (str): Model of the vehicle
    '''

    print(f'Vehicle make: {vehicle_make}\nVehicle model: {vehicle_model}')



# Defining a function to extract all information provided in the original "About Me" prompt
def extract_all_info(name, job_title, num_children, vehicle_make, vehicle_model, company_name, favorite_vg_series):
    '''
    Prints the full "About Me" information

    Inputs:
        - name (str): Name of the person
        - job_title (str): Job title of the person
        - num_chilren (int): The number of children the parent has
        - vehicle_make (str): Make of the vehicle
        - vehicle_model (str): Model of the vehicle
        - company_name (str): Name of the company the person works for
        - favorite_vg_series (str): Person's favorite video game series.
    '''

    print(f'''
    This person\'s name is {name}. Their job title is {job_title}, and they have {num_children} children.
    They drive a {vehicle_make} {vehicle_model}.
    They work for {company_name}.
    Their favorite video game series is {favorite_vg_series}.
    ''')

In [58]:
# Defining how we want ChatGPT to call our custom functions
my_custom_functions = [
    {
        'name': 'extract_person_info',
        'description': 'Get "About Me" information from the body of the input text',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'job_title': {
                    'type': 'string',
                    'description': 'Job title of the person'
                },
                'num_children': {
                    'type': 'integer',
                    'description': 'Number of children the person is a parent to'
                }
            }
        }
    },
    {
        'name': 'extract_vehicle_info',
        'description': 'Extract the make and model of the person\'s car',
        'parameters': {
            'type': 'object',
            'properties': {
                'vehicle_make': {
                    'type': 'string',
                    'description': 'Make of the person\'s vehicle'
                },
                'vehicle_model': {
                    'type': 'string',
                    'description': 'Model of the person\'s vehicle'
                }
            }
        }
    },
    {
        'name': 'extract_all_info',
        'description': 'Extract all information about a person including their vehicle make and model',
        'parameters': {
            'type': 'object',
            'properties': {
                'name': {
                    'type': 'string',
                    'description': 'Name of the person'
                },
                'job_title': {
                    'type': 'string',
                    'description': 'Job title of the person'
                },
                'num_children': {
                    'type': 'integer',
                    'description': 'Number of children the person is a parent to'
                },
                'vehicle_make': {
                    'type': 'string',
                    'description': 'Make of the person\'s vehicle'
                },
                'vehicle_model': {
                    'type': 'string',
                    'description': 'Model of the person\'s vehicle'
                },
                'company_name': {
                    'type': 'string',
                    'description': 'Name of the company the person works for'
                },
                'favorite_vg_series': {
                    'type': 'string',
                    'description': 'Name of the person\'s favorite video game series'
                }
            }
        }
    }
]

In [59]:
# Defining a list of samples
samples = [
    str(about_me),
    'My name is David Hundley. I am a principal machine learning engineer, and I have two daughters.',
    'She drives a Kia Sportage.'
]

In [60]:
# Iterating over the three samples
for i, sample in enumerate(samples):

    print(f'Sample #{i + 1}\'s results:')

    # Getting the response back from ChatGPT (gpt-3.5-turbo)
    openai_response = client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': sample}],
        functions = my_custom_functions,
        function_call = 'auto'
    )

    # Printing the sample's response
    response = openai_response.model_dump_json(indent=2)
    print(response)

Sample #1's results:
{
  "id": "chatcmpl-8njaqRHPPCMQ7VtPhXjPBMsRhcuBA",
  "choices": [
    {
      "finish_reason": "function_call",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "role": "assistant",
        "function_call": {
          "arguments": "{\n  \"name\": \"David Hundley\",\n  \"job_title\": \"principal machine learning engineer\",\n  \"num_children\": 2,\n  \"vehicle_make\": \"Tesla\",\n  \"vehicle_model\": \"Model 3\",\n  \"company_name\": \"State Farm\",\n  \"favorite_vg_series\": \"The Legend of Zelda\"\n}",
          "name": "extract_all_info"
        },
        "tool_calls": null
      }
    }
  ],
  "created": 1706864024,
  "model": "gpt-3.5-turbo-0613",
  "object": "chat.completion",
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 77,
    "prompt_tokens": 320,
    "total_tokens": 397
  }
}
Sample #2's results:
{
  "id": "chatcmpl-8njas4yIuQrp3ZOGSzq9EcGY84U1w",
  "choices": [
    {
      "finish_reas

In [62]:
# Iterating over the three samples
for i, sample in enumerate(samples):

    print(f'Sample #{i + 1}\'s results:')

    # Getting the response back from ChatGPT (gpt-3.5-turbo)

    openai_response = json.loads(client.chat.completions.create(
        model = 'gpt-3.5-turbo',
        messages = [{'role': 'user', 'content': sample}],
        functions = my_custom_functions,
        function_call = 'auto'
    ).model_dump_json(indent=2))['choices'][0]['message']

    # Checking to see that a function call was invoked
    if openai_response.get('function_call'):

        # Checking to see which specific function call was invoked
        function_called = openai_response['function_call']['name']

        # Extracting the arguments of the function call
        function_args = json.loads(openai_response['function_call']['arguments'])

        # Invoking the proper functions
        if function_called == 'extract_person_info':
            extract_person_info(*list(function_args.values()))
        elif function_called == 'extract_vehicle_info':
            extract_vehicle_info(*list(function_args.values()))
        elif function_called == 'extract_all_info':
            extract_all_info(*list(function_args.values()))

Sample #1's results:

    This person's name is David Hundley. Their job title is Principal Machine Learning Engineer, and they have 2 children.
    They drive a Tesla Model 3.
    They work for State Farm.
    Their favorite video game series is The Legend of Zelda.
    
Sample #2's results:
This person's name is David Hundley. Their job title is Principal Machine Learning Engineer, and they have 2 children.
Sample #3's results:
Vehicle make: Kia
Vehicle model: Sportage
